In [1]:
import random

In [2]:
# class Game:

#     #returns total value given cards
#     def total_card_value(self,cards):
#         sum = 0
#         ace_count = 0
#         for card in cards:
#             if card[0] in ['Jack','Queen','King']:
#                 sum += 10
#             elif card[0] == 'Ace':
#                 if sum +11<22:
#                     ace_count += 1
#                     sum += 11
#                 else:
#                     sum += 1
#             else:
#                 sum += int(card[0])
#             #If sum hit 21 later, use aces as 1s not 11s
#             while(sum > 21 and ace_count >1):
#                 sum -=10
#         return sum
#     #simulate move here (for now implement human choices)
#     def make_move(self):
#        return input("1 to play, 2 to stop")
    
#     def play(self):
#         #Set up the deck
#         self.card_categories = ['Hearts','Diamonds','Clubs','Spades']
#         self.cards_list = ['Ace','2','3','4','5','6','7','8','9','10','Jack','Queen','King']
#         deck = [(card,category) for category in card_categories for card in cards_list]
#         random.shuffle(deck)
#         player_card = [deck.pop(),deck.pop()]
#         dealer_card = [deck.pop(),deck.pop()]
#         reward = 0
#         #play the game
#         while True:
#             player_score = 0
#             player_score = total_card_value(player_card)
#             dealer_score = total_card_value(dealer_card)
#             print("Cards Player Has:", player_card)
#             print("Score Of The Player:", player_score)
#             if player_score > 21:
#                 reward =-1
#                 return reward
#             choice = make_move()
#             if choice == '1':
#                 new_card = deck.pop()
#                 player_card.append(new_card)
#             elif choice == '2':
#                 break
#             else:
#                 print("invalid choice try again")
#                 continue
#             print("Player score:",player_score)
            
#         while dealer_score < 17 and reward ==0:
#             new_card = deck.pop()
#             dealer_card.append(new_card)
#             dealer_score = total_card_value(dealer_card)
#         if dealer_score > 21:
#             reward =1
#         elif player_score > dealer_score:
#             reward=1
#         elif dealer_score > player_score:
#             reward=-1
#         else:
#             reward = 0
#         print("Dealer had:", dealer_card)
#         print("reward: ",reward)
#         return reward

In [3]:
import numpy as np
class QLearn:
    def __init__(self, lr=0.1, exp_rate=0.3):
            self.player_Q_Values = {}  # key: [(player_value, show_card, usable_ace)][action] = value
            # initialise Q values | (12-21) x (1-10) x (True, False) x (1, 0) 400 in total
            for i in range(12, 22):
                for j in range(1, 11):
                    for k in [True, False]:
                        self.player_Q_Values[(i, j, k)] = {}
                        for a in [1, 0]:
                            if (i == 21) and (a == 0):
                                self.player_Q_Values[(i, j, k)][a] = 1
                            else:
                                self.player_Q_Values[(i, j, k)][a] = 0
            self.card_categories = ['Hearts','Diamonds','Clubs','Spades']
            self.cards_list = [1,2,3,4,5,6,7,8,9,10,10,10,10]
            self.deck = [card for category in self.card_categories for card in self.cards_list]
            random.shuffle(self.deck)
            
            self.player_state_action = []
            self.state = (0, 0, False)  # initial state
            self.actions = [1, 0]  # 1: HIT  0: STAND
            self.end = False
            self.lr = lr
            self.exp_rate = exp_rate
    def giveCard(self):
        return self.deck.pop()
    def deal2cards(self,show=False):
        card1 = self.giveCard()
        card2 = self.giveCard()
        usableAce = False
        if card1==11 or card2 ==11:
            usableAce = True
        if show:
            return card1+card2,usableAce,card1
        else:
            return card1+card2,usableAce
    
    def dealerPolicy(self, current_value, usable_ace, is_end):
            if current_value > 21:
                if usable_ace:
                    current_value -= 10
                    usable_ace = False
                else:
                    return current_value, usable_ace, True
            # HIT17
            if current_value >= 17:
                return current_value, usable_ace, True
            else:
                card = self.giveCard()
                if card == 1:
                    if current_value <= 10:
                        return current_value+11, True, False
                    return current_value+1, usable_ace, False
                else:
                    return current_value+card, usable_ace, False
    def chooseAction(self):
        # if current value <= 11, always hit
        current_value = self.state[0]
        if current_value <= 11:
            return 1

        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        #             print("random action", action)
        else:
            # greedy action
            v = -999
            action = 0
            for a in self.player_Q_Values[self.state]:
                if self.player_Q_Values[self.state][a] > v:
                    action = a
                    v = self.player_Q_Values[self.state][a]
        #             print("greedy action", action)
        return action
        
    def playerNxtState(self, action):
        current_value = self.state[0]
        show_card = self.state[1]
        usable_ace = self.state[2]

        if action:
            # action hit
            card = self.giveCard()
            if card == 1:
                if current_value <= 10:
                    current_value += 11
                    usable_ace = True
                else:
                    current_value += 1
            else:
                current_value += card
        else:
            # action stand
            self.end = True
            return (current_value, show_card, usable_ace)

        if current_value > 21:
            if usable_ace:
                current_value -= 10
                usable_ace = False
            else:
                self.end = True
                return (current_value, show_card, usable_ace)

        return (current_value, show_card, usable_ace)

        
    def winner(self, player_value, dealer_value):
            # player 1 | draw 0 | dealer -1
            winner = 0
            if player_value > 21:
                if dealer_value > 21:
                    # draw
                    winner = 0
                else:
                    winner = -1
            else:
                if dealer_value > 21:
                    winner = 1
                else:
                    if player_value < dealer_value:
                        winner = -1
                    elif player_value > dealer_value:
                        winner = 1
                    else:
                        # draw
                        winner = 0
            return winner
    
    def winner(self, player_value, dealer_value):
            # player 1 | draw 0 | dealer -1
            winner = 0
            if player_value > 21:
                if dealer_value > 21:
                    # draw
                    winner = 0
                else:
                    winner = -1
            else:
                if dealer_value > 21:
                    winner = 1
                else:
                    if player_value < dealer_value:
                        winner = -1
                    elif player_value > dealer_value:
                        winner = 1
                    else:
                        # draw
                        winner = 0
            return winner
    
    def _giveCredit(self, player_value, dealer_value):
            reward = self.winner(player_value, dealer_value)
            # backpropagate reward
            for s in reversed(self.player_state_action):
                state, action = s[0], s[1]
                reward = self.player_Q_Values[state][action] + self.lr*(reward - self.player_Q_Values[state][action])
                self.player_Q_Values[state][action] = round(reward, 3)

    def game(self):
        self.deck = [card for category in self.card_categories for card in self.cards_list]
        random.shuffle(self.deck)
        # give 2 cards
        dealer_value, d_usable_ace, show_card = self.deal2cards(show=True)
        player_value, p_usable_ace = self.deal2cards(show=False)
        
        self.state = (player_value, show_card, p_usable_ace)
        print("init", self.state)
        
        # judge winner after 2 cards
        if player_value == 21 or dealer_value == 21:
            # game end
            next
        else:
            while True:
                action = self.chooseAction()  # state -> action
                if self.state[0] >= 12:
                    state_action_pair = [self.state, action]
                    self.player_state_action.append(state_action_pair)
                # update next state
                self.state = self.playerNxtState(action)
                if self.end:
                    break
        
                    # dealer's turn
            is_end = False
            while not is_end:
                dealer_value, d_usable_ace, is_end = self.dealerPolicy(dealer_value, d_usable_ace, is_end)
        
            # judge winner
            # give reward and update Q value
            player_value = self.state[0]
            print("player value {} | dealer value {}".format(player_value, dealer_value))
    def train(self):
        for i in range(10000):
            self.game()

In [4]:
game = QLearn()
game.train()

init (11, 10, False)
player value 20 | dealer value 20
init (5, 7, False)
player value 11 | dealer value 17
init (13, 6, False)
player value 23 | dealer value 26
init (9, 10, False)
player value 20 | dealer value 22
init (7, 10, False)
player value 18 | dealer value 20
init (11, 10, False)
player value 21 | dealer value 22
init (20, 5, False)
player value 30 | dealer value 24
init (19, 10, False)
player value 28 | dealer value 23
init (5, 10, False)
player value 8 | dealer value 19
init (20, 3, False)
player value 21 | dealer value 17
init (11, 5, False)
player value 21 | dealer value 23
init (13, 7, False)
player value 23 | dealer value 25
init (13, 2, False)
player value 23 | dealer value 17
init (9, 10, False)
player value 15 | dealer value 21
init (18, 3, False)
player value 24 | dealer value 21
init (11, 3, False)
player value 18 | dealer value 24
init (17, 2, False)
player value 22 | dealer value 22
init (10, 9, False)
player value 14 | dealer value 20
init (9, 9, False)
player v

In [5]:
# game = Game()
# game.play()

In [6]:
!pip install matplotlib
!pip install seaborn
!pip install tqdm
!pip install gymnasium

In [7]:
# #Try Gymnasium
# from __future__ import annotations

# from collections import defaultdict

# import matplotlib.pyplot as plt
# import numpy as np
# # import seaborn as sns
# from matplotlib.patches import Patch
# from tqdm import tqdm

# import gymnasium as gym


# # Let's start by creating the blackjack environment.
# # Note: We are going to follow the rules from Sutton & Barto.
# # Other versions of the game can be found below for you to experiment.

# env = gym.make("Blackjack-v1", sab=True)